<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/Multilayer_QMC_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
import tensorflow as tf
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [3]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


## Baseline

In [4]:
BATCH_SIZE = 256

model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(10),
tf.keras.layers.Softmax()
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)
EPOCHS = 10
  
history = model.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 1s 3ms/step - loss: 0.8602 - categorical_accuracy: 0.7553 - val_loss: 0.2469 - val_categorical_accuracy: 0.9327
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2469 - categorical_accuracy: 0.9304 - val_loss: 0.1915 - val_categorical_accuracy: 0.9467
Epoch 3/10
188/188 [==============================] - 0s 3ms/step - loss: 0.1745 - categorical_accuracy: 0.9499 - val_loss: 0.1550 - val_categorical_accuracy: 0.9560
Epoch 4/10
188/188 [==============================] - 0s 3ms/step - loss: 0.1379 - categorical_accuracy: 0.9612 - val_loss: 0.1333 - val_categorical_accuracy: 0.9620
Epoch 5/10
188/188 [==============================] - 0s 3ms/step - loss: 0.1118 - categorical_accuracy: 0.9698 - val_loss: 0.1217 - val_categorical_accuracy: 0.9644
Epoch 6/10
188/188 [==============================] - 1s 3ms/step - loss: 0.0973 - categorical_accuracy: 0.9733 - val_loss: 0.1138 - val_categorical_accuracy: 0.9660
Epoc

## Full training

In [5]:
BATCH_SIZE = 256
input_dim = 784
num_rff = 512
gamma = 2**-5
n_comp = 80
random_state = 0
dim_h = 30

inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=num_rff , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMClassifSDecompFDMatrix(dim_x=num_rff, dim_y=dim_h, n_comp=n_comp)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMClassifSDecompFDMatrix(dim_x=dim_h, dim_y=10, n_comp=n_comp)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

# fm_x1.trainable = False

EPOCHS = 10
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 14s 71ms/step - loss: 1.4059 - accuracy: 0.6052 - val_loss: 0.4334 - val_accuracy: 0.8520
Epoch 2/10
188/188 [==============================] - 13s 71ms/step - loss: 0.3891 - accuracy: 0.8684 - val_loss: 0.3760 - val_accuracy: 0.8718
Epoch 3/10
188/188 [==============================] - 13s 70ms/step - loss: 0.3196 - accuracy: 0.8852 - val_loss: 0.3456 - val_accuracy: 0.8658
Epoch 4/10
188/188 [==============================] - 13s 71ms/step - loss: 0.2874 - accuracy: 0.8900 - val_loss: 0.3297 - val_accuracy: 0.8795
Epoch 5/10
188/188 [==============================] - 13s 70ms/step - loss: 0.2688 - accuracy: 0.8929 - val_loss: 0.3179 - val_accuracy: 0.8816
Epoch 6/10
188/188 [==============================] - 13s 71ms/step - loss: 0.2562 - accuracy: 0.8962 - val_loss: 0.3091 - val_accuracy: 0.8827
Epoch 7/10
188/188 [==============================] - 13s 71ms/step - loss: 0.2380 - accuracy: 0.9016 - val_loss: 0.3041 - val_accuracy:

## Layerwise training

In [6]:
BATCH_SIZE = 256
input_dim = 784
component_dim = 128
gamma = 2**-5
n_comp = 80
random_state = 0

inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
#qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig, num_eig=num_eig)
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
  
history = qmdmclf.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 8s 42ms/step - loss: 1.6390 - accuracy: 0.4964 - val_loss: 0.5697 - val_accuracy: 0.8938
Epoch 2/10
188/188 [==============================] - 8s 42ms/step - loss: 0.5328 - accuracy: 0.8988 - val_loss: 0.4274 - val_accuracy: 0.9255
Epoch 3/10
188/188 [==============================] - 8s 42ms/step - loss: 0.3972 - accuracy: 0.9301 - val_loss: 0.3680 - val_accuracy: 0.9365
Epoch 4/10
188/188 [==============================] - 8s 41ms/step - loss: 0.3344 - accuracy: 0.9439 - val_loss: 0.3350 - val_accuracy: 0.9425
Epoch 5/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2952 - accuracy: 0.9519 - val_loss: 0.3137 - val_accuracy: 0.9472
Epoch 6/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2704 - accuracy: 0.9568 - val_loss: 0.2934 - val_accuracy: 0.9497
Epoch 7/10
188/188 [==============================] - 8s 42ms/step - loss: 0.2484 - accuracy: 0.9624 - val_loss: 0.2777 - val_accuracy: 0.9523

In [7]:
inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMClassifSDecompFDMatrix(dim_x=10, dim_y=10, n_comp=n_comp)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

# We freeze the weights of the first layer and train the second
fm_x1.set_weights(fm_x.get_weights())
fm_x1.trainable = False
qmdmc1.set_weights(qmdmc.get_weights())
qmdmc1.trainable = False
EPOCHS = 5
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/5
188/188 [==============================] - 11s 53ms/step - loss: 1.0492 - accuracy: 0.9238 - val_loss: 0.2500 - val_accuracy: 0.9600
Epoch 2/5
188/188 [==============================] - 10s 55ms/step - loss: 0.1993 - accuracy: 0.9727 - val_loss: 0.2382 - val_accuracy: 0.9615
Epoch 3/5
188/188 [==============================] - 10s 55ms/step - loss: 0.1867 - accuracy: 0.9752 - val_loss: 0.2278 - val_accuracy: 0.9626
Epoch 4/5
188/188 [==============================] - 11s 56ms/step - loss: 0.1770 - accuracy: 0.9775 - val_loss: 0.2237 - val_accuracy: 0.9640
Epoch 5/5
188/188 [==============================] - 10s 52ms/step - loss: 0.1703 - accuracy: 0.9783 - val_loss: 0.2213 - val_accuracy: 0.9646


In [8]:
# We free all the weights and fine tune

fm_x1.trainable = True
qmdmc1.trainable = True
EPOCHS = 10
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 10s 53ms/step - loss: 0.1661 - accuracy: 0.9786 - val_loss: 0.2182 - val_accuracy: 0.9656
Epoch 2/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1606 - accuracy: 0.9796 - val_loss: 0.2116 - val_accuracy: 0.9663
Epoch 3/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1562 - accuracy: 0.9804 - val_loss: 0.2100 - val_accuracy: 0.9660
Epoch 4/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1512 - accuracy: 0.9812 - val_loss: 0.2074 - val_accuracy: 0.9654
Epoch 5/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1472 - accuracy: 0.9820 - val_loss: 0.2066 - val_accuracy: 0.9663
Epoch 6/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1433 - accuracy: 0.9826 - val_loss: 0.2030 - val_accuracy: 0.9676
Epoch 7/10
188/188 [==============================] - 10s 52ms/step - loss: 0.1399 - accuracy: 0.9835 - val_loss: 0.2011 - val_accuracy: